In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

In [8]:
vocabulary = ['A', 'o', 'e', 't', 'p', 'g', 'k']
heard = str(np.loadtxt('/Users/ash/Downloads/speaker.txt', dtype=np.str))
transition = 1/3 * np.ones(shape=(3,3))
emission = 1/7 * np.ones(shape=(3,7))

In [ ]:
def M_step(alpha, beta, gamma, data, vocab=vocabulary): 
    T = np.zeros(shape=(3,3))
    E = np.zeros(shape=(3,7))
    pairwise = pairwise_marg(alpha, beta)
    
    for i in range(T.shape[0]):
        for j in range(T.shape[1]):
            T[i,j] = np.sum(pairwise, axis=2)
        T[i,:] = T[i,:]/np.sum(T[i,:])
        
    for i in range(E.shape[0]):
        for j in range(E.shape[1]): 
            dumb = [data[_] == vocab[j] for _ in range(len(data))]
            E[i,j] = dumb*gamma[i]
        E[i,:] = E[i,:]/np.sum(E[i,:])
    
    return T, E
                
    
def E_step(T, E, data, vocab=vocabulary):
    #forward pass 
    alpha = np.zeros(shape=(3,len(data)))
    beta = np.zeros(shape=(3,len(data)))
    
    alpha[:, 0] = 1/3
    for i in range(alpha.shape[0]):
        for j in range(1, alpha.shape[1]):
            alpha[i,j] = E[i,j] * np.sum(T[i,:]*alpha[i,j-1])
        alpha[i,:] = alpha[i,:]/np.sum(alpha[i,:])
        
    beta[:, -1] = 1/3
    for i in range(beta.shape[0]):
        for j in range(beta.shape[1]-1, 0, -1):
            beta[i,j-1] = np.sum(E[i,:]*beta[i,j]*T[i,:])
        beta[i,:] = beta[i,:]/np.sum(beta[i,:])
    
    gamma = np.multiply(alpha, beta)
    for _ in range(gamma.shape[0]):
        gamma[_,:] = gamma[_,:]/np.sum(gamma[_,:])
        
    return alpha, beta, gamma

for k in range(10):
    alpha, beta, gamma = E_step(transition, emission, heard)
    transition, emission = M_step(alpha, beta, gamma, heard)